In [1]:
import numpy as np
import pandas as pd

np.__version__, pd.__version__

('1.23.5', '1.5.2')

In [5]:
# select rows in the order of the abs difference to a constant value
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
aValue = 43.0
df.loc[df['CCC'].sub(aValue).abs().argsort()]

,AAA,BBB,CCC
1,5,20,50
0,4,10,100
2,6,30,-30
3,7,40,-50


In [5]:
# creating new columns using applymap
df = pd.DataFrame({"AAA": [1, 2, 1, 3], "BBB": [1, 1, 2, 2], "CCC": [2, 1, 3, 1]})
print(df)
new_cols = [c+"_cat" for c in df.columns]
cat_dict = {1: "a", 2: "b", 3: "c"}
df[new_cols] = df.applymap(cat_dict.get)
df

   AAA  BBB  CCC
0    1    1    2
1    2    1    1
2    1    2    3
3    3    2    1


,AAA,BBB,CCC,AAA_cat,BBB_cat,CCC_cat
0,1,1,2,a,a,b
1,2,1,1,b,a,a
2,1,2,3,a,b,c
3,3,2,1,c,b,a


In [10]:
# get min() of a column for each group
df = pd.DataFrame(
    {"AAA": [1, 1, 1, 2, 2, 2, 3, 3], "BBB": [2, 1, 3, 4, 5, 1, 2, 3]}
)
print(df)
# method 1 using idxmin()
df1 = df.loc[df.groupby('AAA')['BBB'].idxmin()]
print("method 1")
print(df1)
# method 2 using sort and first
df2 = df.sort_values('BBB').groupby('AAA', as_index=False).first()
print("method 2")
print(df2)
# method 3
print(df.groupby('AAA')['BBB'].min().reset_index())
# method 4
print(df.groupby('AAA', as_index=False)['BBB'].min())

   AAA  BBB
0    1    2
1    1    1
2    1    3
3    2    4
4    2    5
5    2    1
6    3    2
7    3    3
method 1
   AAA  BBB
1    1    1
5    2    1
6    3    2
method 2
   AAA  BBB
0    1    1
1    2    1
2    3    2
   AAA  BBB
0    1    1
1    2    1
2    3    2
   AAA  BBB
0    1    1
1    2    1
2    3    2


In [13]:
# Creating a MultiIndex from a labeled frame, then stacking and convert to new columns
df = pd.DataFrame(
    {
        "row": [0, 1, 2],
        "One_X": [1.1, 1.1, 1.1],
        "One_Y": [1.2, 1.2, 1.2],
        "Two_X": [1.11, 1.11, 1.11],
        "Two_Y": [1.22, 1.22, 1.22],
    }
).set_index("row")
print(df)
df.columns = pd.MultiIndex.from_tuples([c.split("_") for c in df.columns])
print(df)
df = df.stack(level=0)
print(df)
df = df.reset_index(level=1).rename(columns={"level_1": "Group_Num"})
print(df)

     One_X  One_Y  Two_X  Two_Y
row                            
0      1.1    1.2   1.11   1.22
1      1.1    1.2   1.11   1.22
2      1.1    1.2   1.11   1.22
     One        Two      
       X    Y     X     Y
row                      
0    1.1  1.2  1.11  1.22
1    1.1  1.2  1.11  1.22
2    1.1  1.2  1.11  1.22
            X     Y
row                
0   One  1.10  1.20
    Two  1.11  1.22
1   One  1.10  1.20
    Two  1.11  1.22
2   One  1.10  1.20
    Two  1.11  1.22
    Group_Num     X     Y
row                      
0         One  1.10  1.20
0         Two  1.11  1.22
1         One  1.10  1.20
1         Two  1.11  1.22
2         One  1.10  1.20
2         Two  1.11  1.22


In [ ]:
# Performing arithmetic with a MultiIndex that needs broadcasting
cols = pd.MultiIndex.from_tuples(
    [(x, y) for x in ["A", "B", "C"] for y in ["O", "I"]]
)
df = pd.DataFrame(np.random.randn(2, 6), index=["n", "m"], columns=cols)
print(df)
df.div(df["C"], level=1) # not working if set level=0

          A                   B                   C          
          O         I         O         I         O         I
n  0.390946  0.200426  0.582800  0.771345 -0.392422  2.363756
m -0.101362 -0.884270 -2.253264 -0.516993 -0.626184 -0.032925


A                    B               C     
          O          I         O          I    O    I
n -0.996239   0.084791 -1.485134   0.326322  1.0  1.0
m  0.161873  26.856898  3.598403  15.702015  1.0  1.0

In [24]:
# Slicing a MultiIndex with xs
coords = [("AA", "one"), ("AA", "six"), ("BB", "one"), ("BB", "two"), ("BB", "six")]
index = pd.MultiIndex.from_tuples(coords)
df = pd.DataFrame([11, 22, 33, 44, 55], index, ["MyData"])
print(df)
print(df.xs("BB", level=0, axis=0)) # level=1 and axis=1 not working for this example
print(df.xs("six", level=1, axis=0)) # level=0 and axis=1 not working for this example

        MyData
AA one      11
   six      22
BB one      33
   two      44
   six      55
     MyData
one      33
two      44
six      55
    MyData
AA      22
BB      55


In [19]:
# Slicing using slice
coords = [("AA", "one"), ("AA", "six"), ("BB", "one"), ("BB", "two"), ("BB", "six")]
index = pd.MultiIndex.from_tuples(coords)
df = pd.DataFrame([11, 22, 33, 44, 55], index, ["MyData"])
print(df)
All = slice(None)
print(df.loc[("BB", All)])  # first level BB, all in second level
print(df.loc[("BB", All),])  # first level BB, all in second level
print(df.loc[(All, "six"),])  # all in first level, six in second level, ##NOTE: must have comma at the end
print(df.loc[(slice('AA','BB'), ["one", "six"]),])  # range in first level and range in second level, ##NOTE: must have comma at the end

        MyData
AA one      11
   six      22
BB one      33
   two      44
   six      55
     MyData
one      33
two      44
six      55
        MyData
BB one      33
   two      44
   six      55
        MyData
AA six      22
BB six      55
        MyData
AA one      11
   six      22
BB one      33
   six      55


In [ ]:
# Flatten multi-level columns
df.columns = df.columns.get_level_values(0) + "_" + df.columns.get_level_values(1)
df.columns = [' '.join(col).strip() for col in df.columns.values]

In [20]:
# flexible apply for dataframe
df = pd.DataFrame(
    {
        "animal": "cat dog cat fish dog cat cat".split(),
        "size": list("SSMMMLL"),
        "weight": [8, 10, 11, 1, 20, 12, 12],
        "adult": [False] * 5 + [True] * 2,
    }
)
print(df)
# List the size of the animals with the highest weight
print(df.loc[df.groupby('animal')['weight'].idxmax(), ['animal','size']])
print(df.groupby('animal').apply(lambda x: x['size'][x['weight'].idxmax()]))

  animal size  weight  adult
0    cat    S       8  False
1    dog    S      10  False
2    cat    M      11  False
3   fish    M       1  False
4    dog    M      20  False
5    cat    L      12   True
6    cat    L      12   True
  animal size
5    cat    L
4    dog    M
3   fish    M
animal
cat     L
dog     M
fish    M
dtype: object


In [ ]:
# flexible apply for dataframe with multiple return values using pd.Series
def f(group):
    return pd.Series([group['weight'].mean(), group['adult'].max(), 0], ['weight','adult','new_column'])
df.groupby(['animal','size']).apply(f)

weight  adult  new_column
animal size                           
cat    L       12.0   True           0
       M       11.0  False           0
       S        8.0  False           0
dog    M       20.0  False           0
       S       10.0  False           0
fish   M        1.0  False           0

In [ ]:
# flexible apply for Series, returning multiple column DataFrame
def f(group):
    return pd.DataFrame({'original': group,
                         'demeaned': group - group.mean()})
print(df.groupby(['animal'])['weight'].apply(f))
print(df.groupby(['animal'], group_keys=True)['weight'].apply(f))#.reset_index(level=0))
print(df.groupby(['animal'], as_index=True)['weight'].apply(f))#.reset_index(level=0)) # as_index not working for series

   original  demeaned
0         8     -2.75
1        10     -5.00
2        11      0.25
3         1      0.00
4        20      5.00
5        12      1.25
6        12      1.25
          original  demeaned
animal                      
cat    0         8     -2.75
       2        11      0.25
       5        12      1.25
       6        12      1.25
dog    1        10     -5.00
       4        20      5.00
fish   3         1      0.00
   original  demeaned
0         8     -2.75
1        10     -5.00
2        11      0.25
3         1      0.00
4        20      5.00
5        12      1.25
6        12      1.25


/tmp/ipykernel_131382/3624195907.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  print(df.groupby(['animal'])['weight'].apply(f))
/tmp/ipykernel_131382/3624195907.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  print(df.groupby(['animal'], as_index=True)['weight'].apply(f))#.reset_index(

In [46]:
# flexible apply for Series, returning multiple column DataFrame
def g(x):
    return pd.Series([x, x**2], ['original', 'squared'])
print(df['weight'].apply(g))

   original  squared
0         8       64
1        10      100
2        11      121
3         1        1
4        20      400
5        12      144
6        12      144


In [ ]:
# Replacing some values with mean of the rest of a group
df = pd.DataFrame({"A": [1, 1, 2, 2], "B": [1, -1, 1, 2], "C": [1, -1, 1, 2]})
print(df)
gb = df.groupby("A")

def replace(group):
    return group.where(group>=0, group[group>=0].mean())

gb.transform(replace) # transform don't modify index, 'A' will not be in the index

   A  B  C
0  1  1  1
1  1 -1 -1
2  2  1  1
3  2  2  2


,index,B,C
0,0,1,1
1,1,1,1
2,2,1,1
3,3,2,2


In [50]:
# Sort groups by aggregated data
df = pd.DataFrame(
    {
        "code": ["foo", "bar", "baz"] * 2,
        "data": [0.16, -0.21, 0.33, 0.45, -0.59, 0.62],
        "flag": [False, True] * 3,
    }
)
print(df)
code_groups = df.groupby("code")
agg_n_sort_order = code_groups[["data"]].transform(sum).sort_values(by="data")
df.loc[agg_n_sort_order.index]

  code  data   flag
0  foo  0.16  False
1  bar -0.21   True
2  baz  0.33  False
3  foo  0.45   True
4  bar -0.59  False
5  baz  0.62   True


,code,data,flag
1,bar,-0.21,True
4,bar,-0.59,False
0,foo,0.16,False
3,foo,0.45,True
2,baz,0.33,False
5,baz,0.62,True


In [52]:
# Select row with maximum value from each group
df = pd.DataFrame(
    {
        "host": ["other", "other", "that", "this", "this"],
        "service": ["mail", "web", "mail", "mail", "web"],
        "no": [1, 2, 1, 2, 1],
    }
)#.set_index(["host", "service"])
print(df)
print(df.loc[df.groupby('host')['no'].idxmax()])

    host service  no
0  other    mail   1
1  other     web   2
2   that    mail   1
3   this    mail   2
4   this     web   1
    host service  no
1  other     web   2
2   that    mail   1
3   this    mail   2


In [59]:
# finding consecutive groups
df = pd.DataFrame([0, 0, 1, 0, 1, 1, 1, 0, 1, 1], columns=["A"])
print(df)
print(df["A"].groupby((df["A"] != df["A"].shift()).cumsum()).groups)

print(df["A"].groupby((df["A"] != df["A"].shift()).cumsum()).cumsum())
print(df["A"].groupby((df["A"] != df["A"].shift()).cumsum()).transform(lambda x: np.max(x.cumsum())))

   A
0  0
1  0
2  1
3  0
4  1
5  1
6  1
7  0
8  1
9  1
{1: [0, 1], 2: [2], 3: [3], 4: [4, 5, 6], 5: [7], 6: [8, 9]}
0    0
1    0
2    1
3    0
4    1
5    2
6    3
7    0
8    1
9    2
Name: A, dtype: int64
0    0
1    0
2    1
3    0
4    3
5    3
6    3
7    0
8    2
9    2
Name: A, dtype: int64


In [61]:
# Create a list of dataframes, split using a delineation based on logic included in rows
df = pd.DataFrame(
    data={
        "Case": ["A", "A", "A", "B", "A", "A", "B", "A", "A"],
        "Data": np.random.randn(9),
    }
)


dfs = list(
    zip(
        *df.groupby(
            (1 * (df["Case"] == "B"))
            .cumsum()
            .rolling(window=3, min_periods=1)
            .median()
        )
    )
)
print(dfs)
print(dfs[-1])

[(0.0, 1.0, 2.0), (  Case      Data
0    A -1.097653
1    A  0.531785
2    A -0.135488
3    B -1.306417,   Case      Data
4    A  0.304657
5    A -2.297835
6    B -0.561967,   Case      Data
7    A -0.255186
8    A  0.077386)]
(  Case      Data
0    A -1.097653
1    A  0.531785
2    A -0.135488
3    B -1.306417,   Case      Data
4    A  0.304657
5    A -2.297835
6    B -0.561967,   Case      Data
7    A -0.255186
8    A  0.077386)


In [ ]:
# pivot table with partial sums and subtotals
df = pd.DataFrame(
    data={
        "Province": ["ON", "QC", "BC", "AL", "AL", "MN", "ON"],
        "City": [
            "Toronto",
            "Montreal",
            "Vancouver",
            "Calgary",
            "Edmonton",
            "Winnipeg",
            "Windsor",
        ],
        "Sales": [13, 6, 16, 8, 4, 3, 1],
    }
)
print(df)
table = pd.pivot_table(
    df,
    values=["Sales"],
    index=["Province"],
    columns=["City"],
    aggfunc=np.sum,
    margins=True,
)
print(table)
print(table.stack("City"))

  Province       City  Sales
0       ON    Toronto     13
1       QC   Montreal      6
2       BC  Vancouver     16
3       AL    Calgary      8
4       AL   Edmonton      4
5       MN   Winnipeg      3
6       ON    Windsor      1
           Sales                                                         
City     Calgary Edmonton Montreal Toronto Vancouver Windsor Winnipeg All
Province                                                                 
AL           8.0      4.0      NaN     NaN       NaN     NaN      NaN  12
BC           NaN      NaN      NaN     NaN      16.0     NaN      NaN  16
MN           NaN      NaN      NaN     NaN       NaN     NaN      3.0   3
ON           NaN      NaN      NaN    13.0       NaN     1.0      NaN  14
QC           NaN      NaN      6.0     NaN       NaN     NaN      NaN   6
All          8.0      4.0      6.0    13.0      16.0     1.0      3.0  51
                    Sales
Province City            
AL       All         12.0
         Calgary      8.

In [65]:

df = pd.DataFrame(
    data={
        "A": [[2, 4, 8, 16], [100, 200], [10, 20, 30]],
        "B": [["a", "b", "c"], ["jj", "kk"], ["ccc"]],
    },
    index=["I", "II", "III"],
)
print(df)

                 A          B
I    [2, 4, 8, 16]  [a, b, c]
II      [100, 200]   [jj, kk]
III   [10, 20, 30]      [ccc]


In [ ]:
def SeriesFromSubList(aList):
    return pd.Series(aList)
df.iloc[0].apply(SeriesFromSubList) # each row is a Series, and convert to DataFrame using apply

,0,1,2,3
A,2,4,8,16.0
B,a,b,c,NaN


In [71]:
df_orgz = pd.concat(
    {ind: row.apply(SeriesFromSubList) for ind, row in df.iterrows()}
)
print(df_orgz)

         0     1     2     3
I   A    2     4     8  16.0
    B    a     b     c   NaN
II  A  100   200   NaN   NaN
    B   jj    kk   NaN   NaN
III A   10  20.0  30.0   NaN
    B  ccc   NaN   NaN   NaN


In [4]:
df = pd.DataFrame(
    {
        'id': [1, 2, 3, 4, 5],
        'name': ['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
        'salary': [70000, 90000, 80000, 60000, 90000],
        'departmentId': [1, 1, 2, 2, 1],
    }
)
print(df)
df1 = pd.DataFrame(
    {
        'id': [1, 2],
        'name': ['IT', 'Sales'],
    }
)
print(df1)

   id   name  salary  departmentId
0   1    Joe   70000             1
1   2    Jim   90000             1
2   3  Henry   80000             2
3   4    Sam   60000             2
4   5    Max   90000             1
   id   name
0   1     IT
1   2  Sales


In [5]:
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    df = employee.loc[employee.groupby('departmentId')['salary'].transform(
        lambda x: x==x.max()
    ),['name','salary','departmentId']]
    #return df
    return df.rename(columns={'name': 'Employee', 'salary': 'Salary'}).merge(
        department,
        left_on='departmentId',
        right_on='id'
    ).rename(columns={'name': 'Department'})[['Department','Employee','Salary']]
department_highest_salary(df, df1)

,Department,Employee,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [6]:
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    df = employee.loc[employee.groupby('departmentId')['salary'].apply(
        lambda x: x.nlargest(1, keep='all').index
    ).explode(),['name','salary','departmentId']]
    #return df
    return df.rename(columns={'name': 'Employee', 'salary': 'Salary'}).merge(
        department,
        left_on='departmentId',
        right_on='id'
    ).rename(columns={'name': 'Department'})[['Department','Employee','Salary']]
department_highest_salary(df, df1)

,Department,Employee,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [7]:
df = pd.DataFrame(
    {
        'id': [1, 2, 3, 4, 5, 6],
        'score': [50, 80, 80, 70, 90, 50],
    }
)
print(df)

def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    return scores.assign(rank=scores['score'].rank(ascending=False,method='dense'))[['score','rank']].sort_values(['rank'])
order_scores(df)

   id  score
0   1     50
1   2     80
2   3     80
3   4     70
4   5     90
5   6     50


,score,rank
4,90,1.0
1,80,2.0
2,80,2.0
3,70,3.0
0,50,4.0
5,50,4.0


In [9]:
# stack
df = pd.DataFrame(
    [[0, 95, 100, 105],[1, 70, None, 80]],
    columns=['product_id', 'store1', 'store2', 'store3']
)
print(df)
def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:
    df = products.set_index('product_id').stack(dropna=True).reset_index()
    df.columns=['product_id','store','price']
    return df
rearrange_products_table(df)

   product_id  store1  store2  store3
0           0      95   100.0     105
1           1      70     NaN      80


,product_id,store,price
0,0,store1,95.0
1,0,store2,100.0
2,0,store3,105.0
3,1,store1,70.0
4,1,store3,80.0


In [10]:
df = pd.DataFrame(
    {
        "student": ['A','B','C','D','E','F','G','H','I'],
        "class": ['Math','English','Math','Biology','Math','Computer','Math','Math','Math']
    }
)
print(df)

  student     class
0       A      Math
1       B   English
2       C      Math
3       D   Biology
4       E      Math
5       F  Computer
6       G      Math
7       H      Math
8       I      Math


In [12]:
ds = df[['class']].value_counts()
ds=ds[ds>=5]
ds

class
Math     6
dtype: int64

In [23]:
for v in ds.index.values:
    print(v[0])

Math


In [28]:
df = pd.DataFrame(
    {
        'sell_date': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-01', '2023-01-02'],
        'product': ['A', 'B', 'A', 'A', 'C']
    }
)
def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
    return activities.drop_duplicates().sort_values(['sell_date','product']).groupby(
        'sell_date'
    ).agg(
        num_sold=('product', 'size'),
        products=('product', ','.join)
    ).reset_index()
categorize_products(df)

,sell_date,num_sold,products
0,2023-01-01,2,"A,B"
1,2023-01-02,2,"A,C"


In [27]:
ds.tolist()

[6]

In [29]:
df_student = pd.DataFrame(
    {
        'student_id': [1,2,13,6],
        'student_name': ['Alice','Bob','John','Alex']
    }
)
df_subject= pd.DataFrame(
    {
        'subject_name':['Math','Physics','Programming']
    }
)
df_student.merge(
        df_subject,
        how='cross'
    )

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [ ]:
# difference between count() and size() after groupby
# count() for other columns, size() for all rows
print('count:')
print(df_subject.groupby('subject_name', group_keys=False).count())
print('size:')
print(df_subject.groupby('subject_name', group_keys=False).size())

count:
Empty DataFrame
Columns: []
Index: [Math, Physics, Programming]
size:
subject_name
Math           1
Physics        1
Programming    1
dtype: int64


In [49]:
df = pd.DataFrame({
    'a': [1,2,3],  
}
)
ds = df.value_counts()
print(ds[ds>0].index.get_level_values(0))

Int64Index([1, 2, 3], dtype='int64', name='a')


In [50]:
# Managers with at least 5 direct reports
df = pd.DataFrame(
    {
        'id': [1,2,3,4,5,6,7,8,9,10,11,12],
        'name': ['A','B','C','D','E','F','G','H','I','J','K','L'],
        'managerId': [3,3,3,3,2,2,4,4,4,4,4,5]
    }
)
print(df)

def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    ds = employee[['managerId']].value_counts()
    return employee.loc[employee['id'].isin(ds[ds>4].index.get_level_values(0)),['name']]
find_managers(df)

    id name  managerId
0    1    A          3
1    2    B          3
2    3    C          3
3    4    D          3
4    5    E          2
5    6    F          2
6    7    G          4
7    8    H          4
8    9    I          4
9   10    J          4
10  11    K          4
11  12    L          5


,name
3,D


In [15]:
import pandas as pd
import numpy as np

pd.Series(5, index=['a','b', 'c'],name='my_series').rename('new_series').loc['a':'c']

a    5
b    5
c    5
Name: new_series, dtype: int64

In [25]:
df=pd.DataFrame({'a':[1,2,3], 'b':4}, index=[0,1,2])
df.insert(1,'c', 8)
df

,a,c,b
0,1,8,4
1,2,8,4
2,3,8,4


In [26]:
df.assign(d=df['a']*2)

,a,c,b,d
0,1,8,4,2
1,2,8,4,4
2,3,8,4,6
